In [1]:
# Modules classiques
import numpy as np
import pandas as pd

# Modules pour data-viz
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.figure_factory as ff

# Options
pd.set_option("display.min_rows", 10)
pd.set_option("display.max_columns", 30)
pd.set_option("max_colwidth", 1000)

sns.set_style('darkgrid')
plt.rc('figure',figsize=(17,13))

# EDA
from wordcloud import *

# Preprocessing 
import re, string
from emoji import demojize

# Modélisation
from sklearn.feature_extraction.text import CountVectorizer

# Autres
from IPython.display import display
from PIL import Image

# Serialization
import pickle

# Distance text
from Levenshtein import distance

# Wordcloud
from nltk.corpus import stopwords
from wordcloud import WordCloud, ImageColorGenerator, get_single_color_func
import cv2
from nltk.stem import WordNetLemmatizer

# Lemmatizer
import spacy
from spacy_lefff import LefffLemmatizer, POSTagger
from spacy.language import Language


from IPython.display import display
import random
import dataframe_image as dfi 

from utils import *

%reload_ext autoreload
%autoreload 2

# Scraping

[Tuto Snscrape](https://betterprogramming.pub/how-to-scrape-tweets-with-snscrape-90124ed006af) <br>
[Tuto Snscrape 2](https://medium.com/dataseries/how-to-scrape-millions-of-tweets-using-snscrape-195ee3594721) <br>
[Dates des émissions](https://fr.wikipedia.org/wiki/Koh-Lanta_:_La_L%C3%A9gende#D%C3%A9roulement)

Scraping effectué à la commande console suivante:

```
snscrape --jsonl --progress --max-results 100000 twitter-search "kohlanta since:2021-08-03 until:2021-08-24" > ./data/emission_0.json
```

Le premier jeu de données concerne les tweets avant la première émission du **24/08**. Les données récupérées sont du 03/08 au 24/08 (exclu). <br>

Ensuite, pour chaque émission, les tweets seront du **jour de l'émission** au **jour de la prochaine émission** (exclu). <br>

Il faudra alors:
1. Supprimer les tweets du jour de l'émission avant **21:05:00**
2. Ajouter les tweets du jour de la prochaine émission avant **21:05:00**

# Fonctions utiles

In [2]:
useful_col = ['id', 'date', 'content', 'user',
              'replyCount', 'retweetCount', 'likeCount', 'quoteCount',
              'lang', 'sourceLabel', # lang pour filtrer ; sourceLabel pour stats
              'media', # Pour récupérer les photos
               # 'retweetedTweet', 
              'quotedTweet', 'inReplyToTweetId', # Savoir si c'est un retweet, une réponse, une quote...
              'mentionedUsers', # Pour repérer les comptes mentionnés le +
              'hashtags' # Pour voir la répartition 
              ]



def clean_dataset(df: pd.DataFrame, col_to_keep : list, since=None, until=None, debug=True):
    '''
    Cette fonction
    1. Conserve uniquement les colonnes utiles
    2. Restreint les tweets à un intervalle défini par [@since, @until]
    3. Supprime les tweets doublons, non français ou encore qui n'ont pas Kohlanta (sous forme de hashtag ou non) dans le contenu de leur tweet
    4. Extrait des informations essentielles 
    '''   
    
    if debug: 
        print('---------- Infos sur le dataset -----------')
        print(f'Taille initiale: {df.shape}')
    
    
    # 1. Conserve uniquement les colonnes utiles
    to_return = df[col_to_keep]
    
    # 2. Restreint les tweets à un intervalle défini par [@since, @until]
    to_return.date = to_return.date.apply(lambda x: x.astimezone('Europe/Paris'))
    tmp_shape = to_return.shape
    
    if since is not None:
        to_return = to_return[to_return.date >= since]
    if until is not None:
        to_return = to_return[to_return.date <= until]
        
    if debug: print(f'{tmp_shape[0] - to_return.shape[0]} tweets ne faisant pas partie de l\'intervalle de temps étudié supprimés.')
        
    # 3. Supprime les tweets doublons
    tmp_shape = to_return.shape
    to_return.drop(index=to_return[to_return.content.duplicated()].index, inplace=True)
    if debug: print(f'{tmp_shape[0] - to_return.shape[0]} duplicats supprimés.')
    
    # 3. Supprime les tweets non français
    tmp_shape = to_return.shape
    to_return.drop(index=to_return[to_return.lang != 'fr'].index, inplace = True)
    to_return.drop(labels='lang', axis='columns', inplace=True)
    
    if debug: print(f'{tmp_shape[0] - to_return.shape[0]} tweets non français supprimés.')
    
    # 3. Certains tweets ne parlent pas de Koh-Lanta mais mentionne quelqu'un qui possède cela dans son username
    tmp_shape = to_return.shape
    tweets_without_kohlanta_content = to_return[~(to_return.content.str.contains('Koh Lanta', case=False) | \
                                    to_return.content.str.contains('KohLanta', case=False) | \
                                    to_return.content.str.contains('Koh-Lanta', case=False) | \
                                    to_return.content.str.contains('Koh- Lanta', case=False) | \
                                    to_return.content.str.contains('Koh -Lanta', case=False))]

    to_return.drop(index=tweets_without_kohlanta_content.index, inplace=True)
    if debug: print(f'{tmp_shape[0] - to_return.shape[0]} tweets avec des usernames incorrects supprimés')
    
    # 4. Extrait des infos essentielles
    # Récupération de l'username
    user_name = to_return.user.apply(func = lambda x: x['username'])
    
    to_return.drop('user', axis='columns', inplace=True)
    to_return['user_name'] = user_name
    
    # Récupération des comptes mentionnés
    def users_mentioned(list_users):
        usernames = []
        for ind_user in range(0, len(list_users)):
            usernames.append(list_users[ind_user]['username'])
        return usernames
        
    tmp = to_return.mentionedUsers.fillna(value=np.nan)
    list_mentioned_users = tmp.apply(lambda x: users_mentioned(x) if type(x) == list else np.nan)
    
    to_return.drop('mentionedUsers', axis='columns', inplace=True)
    to_return['mentioned_users'] = list_mentioned_users
    
    # Le tweet est-il une réponse ?
    isReply = to_return.inReplyToTweetId.apply(lambda x: False if np.isnan(x) else True)
    
    to_return.drop('inReplyToTweetId', axis='columns', inplace=True)
    to_return['isReply'] = isReply
    
    # Le tweet est-il une quote ?
    isQuote = to_return.quotedTweet.apply(lambda x: False if x is None else True)
    
    to_return.drop('quotedTweet', axis='columns', inplace=True)
    to_return['isQuote'] = isQuote
    
    # Extraction de l'URL de la photo s'il en contient une
    def get_img_media_tweets(content):
        if not (content is None): # On vérifie que l'objet n'est pas nul avant d'accéder à son contenu
            if content[0]['_type'] == 'snscrape.modules.twitter.Photo':
                return content[0]['fullUrl']

        return np.nan # Si objet nul ou pas une photo
    
    urlImg = to_return.media.apply(get_img_media_tweets)
    
    to_return.drop('media', axis='columns', inplace=True)
    to_return['urlImg'] = urlImg 
    
    
    if debug:
        print('')
        print(f'Premier tweet: {to_return.date.min()}')
        print(f'Dernier tweet: {to_return.date.max()}')
        print(f'{isReply.sum()} tweets sont des réponses')
        print(f'{isQuote.sum()} tweets sont des quotes')
        print(f'{urlImg.notna().sum()} tweets contiennent des images')
        print('')
        print(f'{df.shape[0]-to_return.shape[0]} lignes supprimées au total')
        print(f'{df.shape[1]-to_return.shape[1]} colonnes supprimées au total')
        print(f'Nouvelle taille: {to_return.shape}')
        print('\n')
    
    return to_return.reset_index(drop=True)

In [3]:
compte_candidats = {
  '@TeheiuraTEAHUI':'teheuira',
  '@Laurentmaistret':'laurent',
  '@NamadiaKohlanta':'namadia',
  '@Blz_Freddy':'freddy',
  '@candicekohlanta':'candice',
  '@alixkohlanta':'alix',
  '@AlexandraKoh':'alexandra',
  '@loickohlanta':'loic',
  '@claude_KohLanta':'claude',
  '@karimakohlanta':'karima',
  '@Coumba_kohLanta':'coumba',
  '@MaximeKohLanta':'maxime',
  '@clemencecastel1':'clemence',
  '@jadekl7':'jade',
  '@DenisBrogniart':'denis',
}

def nlp_pipeline(text):
    '''
    Cette pipeline nettoie nos tweets, on part du moins restrictif (suppression de combinaison particulière) au plus restrictif 
    La pipeline suivie est la suivante:
    0. Remplace les noms des comptes candidats par leur prénom
    1. Passe l'ensemble du texte en minuscule
    2. Remplace les line feed et carriage return par des espaces
    3. Supprime les URLs
    4. Supprime les mentions
    5. Supprime tous les hashtags comment par #koh (insensible à la casse)
    6. Remplace les émoticons par leurs émotions correspondantes
    7. Supprime la ponctuation et les nombres
    
    '''

    for compte, nom in compte_candidats.items():
        text = text.replace(compte, nom)

    text = text.lower() # Passe l'ensemble du texte en minuscule
    text = text.replace('\n', ' ').replace('\r', '') # Remplace les line feed et carriage return par des espaces 
    text = re.sub(r'www\.\S+|https?\S+', '', text) # Supprime les URLs http ou https ou juste www.
    text = re.sub(r'@\S+', '', text) # Supprime les mentions ( Elles peuvent contenir le nom d'un twittos qui est le même que celui d'un candidat !)

    # text = re.sub(r'@\S+', '', text) # Supprime les mentions
    
    text = re.sub(r'#koh\S+', 'koh lanta', text, flags=re.IGNORECASE) # Supprime tous les hashtags commençant par #koh (insensible à la casse)
    text = text.replace('#', '') # Supprime les #
     
#     text = demojize(text, language='fr') # Traite les emojis
#     text = text.replace('_', ' ')

    text = text.replace('é','e').replace('è', 'e').replace('ê', 'e').replace('ï','i') # Supprime les accents
    
#     for p in string.punctuation: # Supprime la ponctuation // text = re.sub(r'[!"#\$%&\'\(\)\*\+,-\.\/:;<=>\?@\[\\\]\^_`{\|}~]', '', text)
#         text = text.replace(p, '')
#     text = re.sub(r'[0-9]', '', text) # Supprime les nombres  

    return text

# Récupération et traitement des données brutes

In [4]:
em0 = pd.read_json('./raw_data/emission_0.json', lines=True)
em1 = pd.read_json('./raw_data/emission_1.json', lines=True)
em2 = pd.read_json('./raw_data/emission_2.json', lines=True)
em3 = pd.read_json('./raw_data/emission_3.json', lines=True)
em4 = pd.read_json('./raw_data/emission_4.json', lines=True)
em5 = pd.read_json('./raw_data/emission_5.json', lines=True)
em6 = pd.read_json('./raw_data/emission_6.json', lines=True)
em7 = pd.read_json('./raw_data/emission_7.json', lines=True)
em8 = pd.read_json('./raw_data/emission_8.json', lines=True)
em9 = pd.read_json('./raw_data/emission_9.json', lines=True)
em10 = pd.read_json('./raw_data/emission_10.json', lines=True)
em11 = pd.read_json('./raw_data/emission_11.json', lines=True)
em12 = pd.read_json('./raw_data/emission_12.json', lines=True)
em13 = pd.read_json('./raw_data/emission_13.json', lines=True)
em14 = pd.read_json('./raw_data/emission_14.json', lines=True)
em15 = pd.read_json('./raw_data/emission_15.json', lines=True)

In [5]:
DEBUG = False

# Avant la première émission
until = pd.Timestamp('2021-08-24 21:05:00', tz='Europe/Paris')
tmp = pd.concat([em0, em1], ignore_index=True)

df_em0 = clean_dataset(df=tmp, col_to_keep=useful_col, until=until, debug=DEBUG)

# Emission 1
since = pd.Timestamp('2021-08-24 21:05:00', tz='Europe/Paris')
until = pd.Timestamp('2021-08-31 21:05:00', tz='Europe/Paris')
tmp = pd.concat([em1, em2], ignore_index=True)

df_em1 = clean_dataset(df=tmp, col_to_keep=useful_col, since=since, until=until, debug=DEBUG)

# Emission 2
since = pd.Timestamp('2021-08-31 21:05:00', tz='Europe/Paris')
until = pd.Timestamp('2021-09-14 21:05:00', tz='Europe/Paris')
tmp = pd.concat([em2, em3], ignore_index=True)

df_em2 = clean_dataset(df=tmp, col_to_keep=useful_col, since=since, until=until, debug=DEBUG)


# Emission 3
since = pd.Timestamp('2021-09-14 21:05:00', tz='Europe/Paris')
until = pd.Timestamp('2021-09-21 21:05:00', tz='Europe/Paris')
tmp = pd.concat([em3, em4], ignore_index=True)

df_em3 = clean_dataset(df=tmp, col_to_keep=useful_col, since=since,until=until, debug=DEBUG)

# Emission 4
since = pd.Timestamp('2021-09-21 21:05:00', tz='Europe/Paris')
until = pd.Timestamp('2021-09-28 21:05:00', tz='Europe/Paris')
tmp = pd.concat([em4, em5], ignore_index=True)

df_em4 = clean_dataset(df=tmp, col_to_keep=useful_col, since=since, until=until, debug=DEBUG)

# Emission 5
since = pd.Timestamp('2021-09-28 21:05:00', tz='Europe/Paris')
until = pd.Timestamp('2021-10-05 21:05:00', tz='Europe/Paris')
tmp = pd.concat([em5, em6], ignore_index=True)

df_em5 = clean_dataset(df=tmp, col_to_keep=useful_col, since=since, until=until, debug=DEBUG)

# Emission 6
since = pd.Timestamp('2021-10-05 21:05:00', tz='Europe/Paris')
until = pd.Timestamp('2021-10-12 21:05:00', tz='Europe/Paris')
tmp = pd.concat([em6, em7], ignore_index=True)

df_em6 = clean_dataset(df=tmp, col_to_keep=useful_col, since=since, until=until, debug=DEBUG)

# Emission 6
since = pd.Timestamp('2021-10-12 21:05:00', tz='Europe/Paris')
until = pd.Timestamp('2021-10-19 21:05:00', tz='Europe/Paris')
tmp = pd.concat([em7, em8], ignore_index=True)

df_em7 = clean_dataset(df=tmp, col_to_keep=useful_col, since=since, until=until, debug=DEBUG)

# Emission 8
since = pd.Timestamp('2021-10-19 21:05:00', tz='Europe/Paris')
until = pd.Timestamp('2021-10-26 21:05:00', tz='Europe/Paris')
tmp = pd.concat([em8, em9], ignore_index=True)

df_em8 = clean_dataset(df=tmp, col_to_keep=useful_col, since=since, until=until, debug=DEBUG)

# Emission 9
since = pd.Timestamp('2021-10-26 21:05:00', tz='Europe/Paris')
until = pd.Timestamp('2021-11-02 21:05:00', tz='Europe/Paris')
tmp = pd.concat([em9, em10], ignore_index=True)

df_em9 = clean_dataset(df=tmp, col_to_keep=useful_col, since=since, until=until, debug=DEBUG)

# Emission 10
since = pd.Timestamp('2021-11-02 21:05:00', tz='Europe/Paris')
until = pd.Timestamp('2021-11-09 21:05:00', tz='Europe/Paris')
tmp = pd.concat([em10, em11], ignore_index=True)

df_em10 = clean_dataset(df=tmp, col_to_keep=useful_col, since=since, until=until, debug=DEBUG)

# Emission 11
since = pd.Timestamp('2021-11-09 21:05:00', tz='Europe/Paris')
until = pd.Timestamp('2021-11-23 21:05:00', tz='Europe/Paris')
tmp = pd.concat([em11, em12], ignore_index=True)

df_em11 = clean_dataset(df=tmp, col_to_keep=useful_col, since=since, until=until, debug=DEBUG)

# Emission 12
since = pd.Timestamp('2021-11-23 21:05:00', tz='Europe/Paris')
until = pd.Timestamp('2021-11-30 21:05:00', tz='Europe/Paris')
tmp = pd.concat([em12, em13], ignore_index=True)

df_em12 = clean_dataset(df=tmp, col_to_keep=useful_col, since=since, until=until, debug=DEBUG)

# Emission 13
since = pd.Timestamp('2021-11-30 21:05:00', tz='Europe/Paris')
until = pd.Timestamp('2021-12-07 21:05:00', tz='Europe/Paris')
tmp = pd.concat([em13, em14], ignore_index=True)

df_em13 = clean_dataset(df=tmp, col_to_keep=useful_col, since=since, until=until, debug=DEBUG)

# Emission 14
since = pd.Timestamp('2021-12-07 21:05:00', tz='Europe/Paris')
until = pd.Timestamp('2021-12-14 21:05:00', tz='Europe/Paris')
tmp = pd.concat([em14, em15], ignore_index=True)

df_em14 = clean_dataset(df=tmp, col_to_keep=useful_col, since=since, until=until, debug=DEBUG)

# Emission 15
since = pd.Timestamp('2021-12-14 21:05:00', tz='Europe/Paris')

df_em15 = clean_dataset(df=em15, col_to_keep=useful_col, since=since, debug=DEBUG)

In [7]:
dfs = [df_em0, df_em1, df_em2, df_em3, df_em4, df_em5, df_em6, df_em7, df_em8, df_em9, df_em10, df_em11, df_em12, df_em13, df_em14, df_em15]

# Création des datasets

## NER (Détection des noms des candidats)

Ici, je repère l'ensemble des entités de personnes dans les tweets pour pouvoir déduire les appelations données à l'ensemble des candidats. <br>
Ensuite j'effectue une distance de similarité entre ces "appelations" et les noms réels des candidats. <br>
Cela me permet d'assigner l'ensemble des appelations à un candidat donné. <br>

Voir: https://www.baeldung.com/cs/string-similarity-edit-distance

In [14]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner")
model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner")

nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="simple")

In [ ]:
def trouve_surnoms(df:pd.DataFrame, nom_candidats: list):
    
    df.content = df.content.apply(nlp_pipeline) # Nettoie notre corpus
       
    # Par batch de 512, récupère l'ensemble des entités "Personnes" trouvées dans le contenu des tweets
    persons = []
    start = 0
    for stop in range(0, df.shape[0], 512):
        output = nlp(' '.join(df.iloc[start:stop].content.values)) # Concatène 512 tweets en un seul string
        start = stop
        for entity in output:
            if entity['entity_group'] == 'PER':
                persons.append(entity['word']) # Ajoute l'entité reconnue à notre liste
      
    # On nettoie ici les noms/surnoms obtenus
    persons = list(map(lambda x: x.split(), persons)) # On éclate en un unique mots les entités contenant plusieurs mots (Ex: 'Claude le meilleur')
    clean = [] # clean est notre liste avec tous les surnoms trouvés écrits 'proprement'
    for nom in persons: # On passe notre liste en 1D
        if type(nom)==list:
            clean += nom
        else:
            clean.append(nom)
            
    clean = [nom for nom in clean if (len(nom) >= 3 and len(nom) < 13)] # On supprime les noms trop courts ou trop longs
    clean = np.unique(clean)
    
    # On calcule la distance de levenshtein entre les surnoms trouvés et les noms réels des candidats.
    # Pour ensuite classer dans un dict les trouvailles
    surnoms_trouves = {}
    for nom in nom_candidats:
        res = [] # res contient l'ensemble des distances noms - surnoms
        distance_max = round(len(nom)*0.33 - 1) # La distance de levenshtein maximum acceptée pour considérer cela comme une faute de frappe/Une autre appellation du candidat
        
        for entity in clean: # Calcul de la distance pour chaque entité trouvée
            res.append(distance(entity, nom.lower()))
        res = pd.Series(res)
        
        surnoms_trouves[nom] = clean[res[res <= distance_max].index] # Assigne les surnoms < à une certaine distance définie aux noms des candidats
        surnoms_trouves[nom] = np.append(surnoms_trouves[nom], nom.lower()) # Ajoute le nom du candidat à cette liste
        
        if nom == 'Teheuira':
            surnoms_trouves[nom] = np.append(surnoms_trouves[nom], 'tehe') # Exception pour teheuira où l'on connait un de ces surnoms populaire
        
        surnoms_trouves[nom] = np.unique(surnoms_trouves[nom]) # On supprime les doublons
                
    return surnoms_trouves

In [20]:
trouve_surnoms(df=df_em5, nom_candidats=NOMS_CANDIDATS)

{'Freddy': array(['freddy'], dtype='<U10'),
 'Teheuira': array(['tehe', 'teheiura', 'teheuira', 'teheura', 'tehueira', 'tehuira',
        'tehura'], dtype='<U10'),
 'Patrick': array(['patrick'], dtype='<U10'),
 'Alexandra': array(['alexandra'], dtype='<U10'),
 'Cindy': array(['cindy'], dtype='<U10'),
 'Claude': array(['claude'], dtype='<U10'),
 'Karima': array(['karima'], dtype='<U10'),
 'Jade': array(['jade'], dtype='<U10'),
 'Maxime': array(['maxime'], dtype='<U10'),
 'Laurent': array(['laurent'], dtype='<U10'),
 'Candice': array(['candice'], dtype='<U10'),
 'Sam': array(['sam'], dtype='<U10'),
 'Clementine': array(['clementine'], dtype='<U10'),
 'Ugo': array(['ugo'], dtype='<U10'),
 'Namadia': array(['amadia', 'namadia', 'namandia'], dtype='<U10'),
 'Christelle': array(['christelle', 'cristelle'], dtype='<U10'),
 'Coumba': array(['coumba', 'coumva', 'cumba', 'koumba'], dtype='<U10'),
 'Loic': array(['loic'], dtype='<U10'),
 'Clemence': array(['clemence'], dtype='<U10'),
 'Phil': arr

In [9]:
def save_all_surnames(from_em=0, until_em=15, dfs=dfs, nom_candidats=nom_candidats):
    '''
    Sauvegarde tous les surnoms des candidats dans un pickle
    '''
    for num in range(from_em, until_em+1):
        path_dict = f'./data_jugement/surnoms/surnoms_em{num}.pickle'
        df = dfs[num] # Récupère le dataset de l'émission
        
        
        d = trouve_surnoms(df=df, nom_candidats=nom_candidats)
        with open(path_dict, 'wb') as f:
            pickle.dump(d, f) # Sauvegarde l'ensemble des surnoms trouvés pour l'émission @num

In [23]:
save_all_surnames(from_em=14, until_em=15, dfs=dfs, nom_candidats=NOMS_CANDIDATS)

In [5]:
from utils import surnoms_trouves

for nom in NOMS_CANDIDATS:
    all_results, result = surnoms_trouves(nom, 15)
    print(all_results)

['freddy']
['teheirua', 'tehehura', 'theura', 'tehura', 'teheuira', 'teheiura', 'tehe', 'theheuira', 'teheiru', 'teheura', 'tehiura', 'tehoura', 'tehuira', 'theuira', 'teheiuira', 'heuira', 'tehueira', 'teiheura']
['patrik', 'patrick']
['alexandra', 'lexandra']
['cindy']
['claude', 'claud']
['karima', 'karim', 'karma']
['jade']
['maxime', 'maxim', 'maxine']
['laurent', 'laurant']
['candice']
['sam']
['clementine']
['ugo']
['namadia', 'namandia', 'amadia']
['christelle', 'cristelle', 'christel']
['coumva', 'comba', 'koumba', 'couba', 'cumba', 'coumba']
['loic']
['clemence', 'clement']
['phil']
['alix']
['donis', 'dennis', 'denis']


## Création de datasets par candidats

La cellule ci-dessous nous créé l'ensemble des datasets pour chaque candidats et pour chaque émission. <br>
Etapes: 
1. Récupère tous les surnoms trouvés pour chaque candidat (fichiers pickle créés dans save_all_surnames)
2. Création du dataset du candidat en fonction des surnoms trouvés
3. Sauvegarde des datasets localement pour une réutilisation ultérieure !

In [14]:
#TODO: Sauver sous un autre format que .csv (trop lent)
# Pickle --> best (+ rapide ecriture / )
# (voir https://towardsdatascience.com/stop-using-csvs-for-storage-here-are-the-top-5-alternatives-e3a7c9018de0)

def save_datasets_candidates(dfs, nom_candidats, from_em=0, until_em=15):
    for num in range(from_em, until_em+1): # On parcourt les datasets
        for nom in nom_candidats: # On parcourt les candidats
            path_to_save = f'./data_jugement/em{num}/'
            path_dict = f'./data_jugement/surnoms/surnoms_em{num}.pickle'
            end_file = f'{nom.lower()}.pickle'
            
            df = dfs[num] # Récupère le dataset de l'émission
            df.content = df.content.apply(nlp_pipeline)
            
            all_results, result = surnoms_trouves(nom, until_em) # Récupère tous les surnoms trouvés pour un candidat
            pattern = '|'.join(list(map(lambda x: '\\b'+x+'\\b', all_results))) # Pattern à respecter dans la recherche des surnoms
            reg_pattern = re.compile(pattern)
            
            to_save = df[df.content.str.contains(reg_pattern, regex=True)] # Récupère tous les tweets mentionnant @nom
            to_save.to_pickle(path_to_save + end_file) # Sauvegarde le nouveau dataset


In [16]:
# 1h20 avec .csv
# 1min35 avec pickle

In [15]:
%%time
save_datasets_candidates(dfs, NOMS_CANDIDATS, 0, 15)

Wall time: 1min 35s
